In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_Y = pd.read_csv("train_2016_v2.csv", index_col = 'parcelid')
train_Y = train_Y[~train_Y.index.duplicated(keep='first')]

In [73]:
allX = pd.read_csv("properties_2016.csv", dtype={"hashottuborspa": bool, "propertycountylandusecode": str,\
                                                    "propertyzoningdesc": str, "fireplaceflag": bool,\
                                                   "taxdelinquencyflag": str}, index_col = 'parcelid')

C:\Users\Ivan\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
train_X = allX.loc[train_Y.index]

In [5]:
def createDataSets():
    dataSets = {}
    # dataSets_Y = {}
    for x in train_X.columns[1:]:
        currentSeriesX = train_X[x].dropna()
        currentSeriesY = train_Y.loc[currentSeriesX.index]['logerror']
        dataSets["{0}".format(x)] = pd.DataFrame(currentSeriesX).join(pd.DataFrame(currentSeriesY))
    return dataSets

In [6]:
dataSets = createDataSets()

In [8]:
actualMean = np.mean(train_Y['logerror'])
actualStdDev = np.std(train_Y['logerror'])
print(actualMean)
print(actualStdDev)

0.011417672767609062
0.16097460590078036


In [9]:
avgErrorDeltas = {}
for key in dataSets:
    if dataSets[key][key].dtype == 'float64':
        avgErrorDeltas[key] = np.mean(dataSets[key]['logerror']) - actualMean
    else:
        pass

In [63]:
avgErrorDF = pd.DataFrame.from_dict(avgErrorDeltas, orient='index')

In [11]:
submission = pd.read_csv("sample_submission.csv", index_col='ParcelId')

C:\Users\Ivan\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
np.random.seed(777)
predictions = np.random.normal(actualMean, actualStdDev, len(submission))
predictions = predictions.round(4)

In [13]:
for cols in submission.columns[1:]:
    submission[cols] = predictions

In [75]:
#%%timeit
for cols in allX.columns:
    #print(cols)
    try:
        allX.loc[allX[cols] > -200000000, cols] = avgErrorDeltas[cols]
    except KeyError:
        print(cols)

airconditioningtypeid
hashottuborspa
propertycountylandusecode
propertyzoningdesc
fireplaceflag
taxdelinquencyflag


In [78]:
allX['sum'] = allX.sum(axis=1)

In [80]:
cachedAllXSum = allX['sum']
cachedAllXSum.to_csv("cachedSum.csv")

In [82]:
try:
    allX['result'] = allX['sum'] + predictions
except TypeError:
    pass

In [92]:
for cols in submission.columns:
    submission[cols] = allX['result']

In [ ]:
submission.round(4)

In [95]:
submission.to_csv("submission2.csv")